<a href="https://colab.research.google.com/github/dolinpatel451/dolinpatel451/blob/master/CS890BN_Programming_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


Dolinkumar Patel

200 392 950

CS 890BN


---



# **Key Chain using RC4-BHF Implementation**


In [1]:
# Adding library to deal with bits and bytes
!pip install bitarray
import bitarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The Appending process

Step 1: Append the padding bits

> Input: input message (variable length)

> Output: 1 or more message blocks (512 bits each)

Process explanation: For a variable length input, we need to convert the input length to a multiple of 512. Even if the input is in multiple of 512, we will still append an entire block behind to ensure the hash function produces a unique output for each possible input.

We append by adding a '1' followed by the required number of zeros.

In [2]:
# A function to implement the appending process 
# 
# Input: Message string or key
# 
# Output: A message in binary with padded blocks and total length of the message

def appending_process(msg):

  # Convert message to a binary format
  msg_binary = ''.join(format(i, 'b') for i in bytearray(msg, 'utf-8'))
  
  # Get length of the message
  n = len(msg_binary)

  # convert length in binary format
  n_in_binary = '{0:016b}'.format(n)

  # Calculate number of padding bits required
  length_msg = len(msg_binary)
  padding_bits_needed = (512 - length_msg) % 512
  total_padding_bits_needed = padding_bits_needed - 16

  # Display the details
  "Number of padding bits needed = " + str(total_padding_bits_needed)

  # create a padding block of 512 bits
  padding_block = '1'
  i = 0
  for i in range(495):
    padding_block = padding_block + '0'
  padding_block += n_in_binary

  padded_message = msg_binary

  # check if no padding bits are needed. If no, we will add an entire block
  if padding_bits_needed == 0:
    padded_message = msg_binary + padding_block
  else:
    for i in range(total_padding_bits_needed):
      padded_message = padded_message + padding_block[i]
    padded_message +=  n_in_binary

  padded_message 
  padded_message_length = len(padded_message)

  # "Length of padded message: " + str(len(padded_message)) + " l: " + str(len(padded_message)/512)
  # print("Appending Process: N + 16 + 1 + v = 0 mod 512\n\n")

  # print("M (N bits) = " + msg_binary + "\nlength of M (16-bits): " + n_in_binary + "\n1\n" + "v: " + padding_block[1:total_padding_bits_needed])

  # print("(N+16+1+v) % 512 = " + str(len(padded_message)%512))

  return padded_message,padded_message_length

    


**The Dividing Process**






In [3]:
# The dividing process
# create message blocks of 512 bits from padded message.
# 
# Input: A padded message and the length of the message in bits
# 
# Output: A list with divided message blocks and total number of message blocks

def dividing_process(padded_message, padded_message_length):
  i = 0
  message_blocks = []
  no_of_blocks = padded_message_length/512
  # no_of_blocks
  for i in range(int(no_of_blocks)):
    block = padded_message[i*512:512*(i+1)]
    message_blocks.append(block)

  ########## FOR PRINTING PURPOSES #############
  # print("The dividing process \n")
  # for i in range(int(no_of_blocks)):
  #   print("M" + str(i) + ": " + message_blocks[i])
  #   print("Length of M" + str(i) + ": " + str(len(message_blocks[i])) + "\n")

  return message_blocks, no_of_blocks

**The Compression Process**

Step 2: Compression

In [4]:
# A function to swap two numbers
def swap(str, i, j):

  # Create a new list
   list1 = list(str)

  # Swap the values
   list1[i], list1[j] = list1[j], list1[i]

  # Return the new list
   return list1

In [5]:
# RC4: KSA implementation
def ksa(msg_block):
  state = []
  t = []
  # print("RC4 m: " + message_block[3])
  # print(len(message_block[0]))
  for i in range(256):
    state.append(str(i))
    t.append(msg_block[i])
  j = 0
  for i in range(256):
    # print(i, ": ", j, state[i], t[i])
    # print(state)
    # print("After swap")
    j = (j + int(state[i]) + int(t[(i)])) % 256 # Compute j
    state = list(swap(state, i, j)) # Swap values at index i & j
    # print(state)

  return state


In [6]:
# RC4: KSA* implementation
def ksa_asterisk(mk, state):
  j = 0
  for i in range(256):
    # print(i, ": ", j, state[i], t[i])
    # print(state)
    # print("After swap")
    j = (j + int(state[i]) + int(mk[(i%64)])) % 256 # Compute j
    state = list(swap(state, i, j)) # Swap values at index i & j
  return state

In [7]:
# PRGA* implementation
# input: length and state(k)
# output: state(mk)

def prga(length, state):
  j = 0

  # Loop to iterate till length
  for i in range(length):
    i = (i + 1) % 256
    j = (j + int(state[i])) % 256 # Compute j
    # print(i, ": ", j, state[i])
    # print(state)
    # print("After swap")
    state = list(swap(state, i, j)) # Swap values at index i & j
    # print(state)
    # print("\n")
  return state


In [8]:
# A function to calculate len
# 
# Input: A block M (512-bit)
# 
# Output: Integer equivalent of the binary value of the block M

def cal_len(mk):

  # Convert string to byte
  mk_bytes = str.encode(mk)

  # Convert Byte to Integer
  mk_int = int(mk_bytes,2) 

  # Calculate len by performing modulo 256
  length = mk_int % 256

  # If the len = 0, the len = offset
  if length == 0:
    return offset
  else:
    return length


In [9]:
# The Compression step
# 
# Input: Message blocks (512-bits each) and number of blocks 
# 
# Output: Final state_n
# 
def compression_step(message_blocks, no_of_blocks):

  state_m = [] # List to store STATE_M (output of KSA*)
  state_n = [] # List to store STATE_N (output of PRGA*)

  # m1 to KSA
  state_m0 = ksa(message_blocks[0])

  #calculate len for m1
  len = cal_len(message_blocks[0])

  # PRGA* for m1
  state_m.append(prga(offset, state_m0))

  # PRGA* for m1 -->input state_m[0]
  state_n.append(prga(len, state_m[0]))

  # loop for the compression step after STATE1

  for i in range(int(no_of_blocks) -1):

    # Perform KSA*
    state_m.append(ksa_asterisk(message_blocks[i+1], state_n[i]))

    # Calculate len
    len = cal_len(message_blocks[i+1])

    # print(state_m[i+1])

    # Perform PRGA*
    state_n.append(prga(len, state_m[i+1]))
  
  # Return the final state value
  return state_n




**The Output Step**

Step 3: Output

In [10]:
# Function to implement the output step
# 
# Input: The last state after PRGA*
# 
# Output: 128-bit hash value
# 
def output(state_n):

  ############ First part ########################

  # Run KSA
  state_N = ksa(state_n[-1])

  # Perform PRGA
  state_final = prga(offset, state_N)

  # Save the last 256 bits and disgard the first 256 bits.
  state_final_256 = state_final[:256]

  # Variables to store the values in binary
  state_final_256_in_binary = []
  state_N_256_in_binary = []
  O_256 = []

  # Loop to convert int to binary
  for i in state_final_256:
    state_final_256_in_binary.append(('{0:08b}'.format(int(i))))


  # Save the last 256 bits and disgard the first 256 bits.
  state_N_256 = state_N[:256]

  # Loop to convert int to binary
  for i in state_N_256:
    state_N_256_in_binary.append(('{0:08b}'.format(int(i))))


  # Loop to perform the XOR operation
  c = 0
  for i in range(256): # 256 bit operations

    # Loop to get individual bit and perform XOR operation
    for a,b in zip(state_final_256_in_binary[i],state_N_256_in_binary[i]):
      c += 1

      # O_256 stores the result of the XOR
      O_256.append((ord(a) ^ ord(b))) # XOR on each bit
  
  ############### Part 2 #############################

  # Converting 256 bytes to 128 bits

  final_hash = '' # variable to store the 128-bit hash
  c = 0

  # Loop to iterate over the 256 bytes stored in O_256 variable
  for i in O_256:

    # Condition to access the 8th bit
    if (c%8 == 0):

      # Condition to ignore the even Byte 
      if (c%16 != 0):

        last_bit = i # store the last bit in a variable
        
        # append the bit to the final hash
        final_hash += str(last_bit)

    c = c + 1
  
  # return the 128-bit hash value
  return final_hash

In [11]:
# Function to convert Binary to Hexadecimal
# It returns hex value without the prefix '0x'
# Input: 128-bit binary
# Output: 16 characters hexadecimal key

def binToHexa(n):
   
    # convert binary to int
    num = int(n, 2)
     
    # convert int to hexadecimal
    hex_num = hex(num)

    return(hex_num[2:])

Code to perform 500 key generation.

**Use Case**

>The key is generated as K[i+1] = RC4-BHF(K[i]). We generate K[n], K[n-1], K[n-2], K[n-3] .... K[1]. But we distribute K[1], K[2],... K[n].

>Since the key generation is a one-way function, it will be impossible to predict the next key. 

>The user can always verify the new key based on the previous key


In [12]:
# Function to generate a key based on RC4-BHF algorithm
# Input: Message and Offset
# Output: 128-bit key in hexadecimal

def rc4_bhf(k, offset):
  print("Input: " + k)

  # call the padding function
  padded_message, padded_message_length = appending_process(k)

  # call the dividing function
  message_blocks, no_of_blocks = dividing_process(padded_message, padded_message_length)

  # Call the compression function
  state_n = compression_step(message_blocks, no_of_blocks)

  # call the output function
  final_hash = output(state_n)

  # Convert 128-bit binary to hexadecimal
  key = binToHexa(final_hash)

  print("Key: " + str(key))
  print("\n")

  # return key in hexadecimal
  return key

Input: 



> Message: 655536 bits


> Offset: positive integer


Output: 128 bit hash value in hexadecimal

In [13]:
########## MAIN BLOCK OF CODE ##################################
# 
# 
# input message
# message = 'Network security is not a joke'
message = 'In the digital realm, we must be aware Of the dangers that lurk everywhere Our networks and systems must be secure To keep our data safe, we must endure'

# Offset is used in PRGA when len = 0
offset = 200

# List to store all the keys
keys = []

# Loop to generate 500 keys
for i in range(500):

  # The first time RC4_BHF function takes a message as input
  if i == 0:
    keys.append(rc4_bhf(message, offset))

  # Rest of the time RC4_BHF function takes the last key as input
  else:
    keys.append(rc4_bhf(keys[i-1], offset))

########## END OF MAIN BLOCK OF CODE ##################################



Input: In the digital realm, we must be aware Of the dangers that lurk everywhere Our networks and systems must be secure To keep our data safe, we must endure
Key: 5f312f1bd242649f5aa71944d0334044


Input: 5f312f1bd242649f5aa71944d0334044
Key: ac0edb1a18d5b7b448b7ca9c4020605c


Input: ac0edb1a18d5b7b448b7ca9c4020605c
Key: 8014d6aa4110a60d82da1213304620a2


Input: 8014d6aa4110a60d82da1213304620a2
Key: 523c1efdc18ed3e5a64b83ea2a0a0611


Input: 523c1efdc18ed3e5a64b83ea2a0a0611
Key: 77ce989dff3947aa5ddbc32ba5c8510b


Input: 77ce989dff3947aa5ddbc32ba5c8510b
Key: 4c4152d1beb305be09262b4e689036e9


Input: 4c4152d1beb305be09262b4e689036e9
Key: 19c4d9a606da3d1d99cc45f8cc220243


Input: 19c4d9a606da3d1d99cc45f8cc220243
Key: 9a37e591cf1667b7a8aa1bcf4a6a000d


Input: 9a37e591cf1667b7a8aa1bcf4a6a000d
Key: 5460c63967f40e91c5169bbcf8106c00


Input: 5460c63967f40e91c5169bbcf8106c00
Key: 2b2ea02bb9f963cde7bb8dbfd4222021


Input: 2b2ea02bb9f963cde7bb8dbfd4222021
Key: cfedfa9cb554357028c031afba420812




In [14]:
# Code to save all the keys in a text file "RC4_BHF_keys.txt"
with open('RC4_BHF_keys.txt', 'w') as writefile:
    writefile.write(str(keys))


In [15]:
# display all 500 keys
keys

['5f312f1bd242649f5aa71944d0334044',
 'ac0edb1a18d5b7b448b7ca9c4020605c',
 '8014d6aa4110a60d82da1213304620a2',
 '523c1efdc18ed3e5a64b83ea2a0a0611',
 '77ce989dff3947aa5ddbc32ba5c8510b',
 '4c4152d1beb305be09262b4e689036e9',
 '19c4d9a606da3d1d99cc45f8cc220243',
 '9a37e591cf1667b7a8aa1bcf4a6a000d',
 '5460c63967f40e91c5169bbcf8106c00',
 '2b2ea02bb9f963cde7bb8dbfd4222021',
 'cfedfa9cb554357028c031afba420812',
 'a617522806a0ae6f65d1b7e399302606',
 '63128344ac5ae887b7be1ed8d0091820',
 '6531760de35bdc5fd63ff23871223439',
 '5c8b575819bb44a18bb5bb3d78709031',
 '4e8c0776d2529246d9e9ff7448904104',
 '2133f25b64728a7d91659579a421b1a4',
 'd997ffb2daf328d0dda00b1df1144000',
 'c054fc8f5daa1cde452a3258019d828',
 '3f9d620688c0e602eb767bb580780c80',
 '7ad7d4453e0afb9987c1eec05ae26392',
 '8b09830a92f551c6c9a7377ed82a0e21',
 'abdce817708dedea56778ff7e87cb4a8',
 'e007fb032c8e954df3308ae17002d890',
 'c14ea18e02fcc71e37cedf8c89814420',
 '7dfa1b4f15eb331e0b2ad695b24010aa',
 'b160a84d1f79ecf5bf80d0732810400c',
 '